In [1]:
from os.path import abspath
import os
import pickle
import pandas as pd
import seaborn as sns
import numpy as np
from scipy.stats import linregress
from scipy.stats import pearsonr
from tqdm.notebook import tqdm
import re

In [2]:
working_dir = "/om2/user/ckauf/.result_caching/neural_nlp.score"

In [3]:
categories_control = ['original',
                      'length-control',
                      'constant-control']

categories_scrambled = ['original',
                        'Scr1',
                        'Scr3',
                        'Scr5',
                        'Scr7',
                        'lowPMI',
                        'lowPMI-random',
                        'backward',
                        'random-wl',
                        'random-wl-samepos']

categories_perturb_loss = ['original',
                           'nouns',
                           'nounsverbs',
                           'nounsverbsadj',
                           'contentwords',
                           'random-nouns',
                           'functionwords']

categories_perturb_meaning = ['sentenceshuffle_passage',
                              'sentenceshuffle_topic',
                              'sentenceshuffle_random']
                              #'sentenceshuffle_random-topic-criteria',
                              #'sentenceshuffle_random-topic-length-criteria']

categories_perturb = categories_perturb_loss + categories_perturb_meaning

categories = [categories_perturb, categories_scrambled, categories_control]
keys = ["perturb", "scrambled", "control"]

categories_dict = dict(zip(keys, categories))

In [4]:
#example structure
with open(os.path.join(working_dir,"benchmark=Pereira2018-encoding-scrambled-random-wl-samepos,model=gpt2,subsample=None.pkl"), "rb") as f:
    df = pickle.load(f)
df

Loading lookup from /om2/user/ckauf/anaconda/envs/perturbedenv/lib/python3.6/site-packages/brainio_collection/lookup.csv


{'data': <xarray.Score (layer: 13, aggregation: 2)>
 array([[0.005489, 0.101422],
        [0.394926, 0.064149],
        [0.502185, 0.116105],
        [0.432593, 0.097443],
        [0.331035, 0.098538],
        [0.306578, 0.094   ],
        [0.307025, 0.059281],
        [0.291988, 0.040568],
        [0.379571, 0.06638 ],
        [0.403858, 0.115814],
        [0.448345, 0.063882],
        [0.55929 , 0.019229],
        [0.626428, 0.070506]])
 Coordinates:
   * layer        (layer) object 'drop' 'encoder.h.0' ... 'encoder.h.11'
   * aggregation  (aggregation) <U6 'center' 'error'
 Attributes:
     raw:          <xarray.Score (layer: 13, aggregation: 2)>\narray([[0.00174...
     ceiling:      <xarray.Score (aggregation: 3)>\narray([0.318567, 0.012958,...
     description:  ceiling-normalized score
     model:        gpt2
     benchmark:    Pereira2018-encoding-scrambled-random-wl-samepos}

In [5]:
def get_max_score(matrix):
    """
    input: result = out['data'].values matrix (e.g. for distilgpt2 a matrix of dimensions 7x2)
    output: maximum score and associated error for this matrix.
    """
    max_score, error = 0,0
    idx = 0
    for i in range(len(matrix)):
        if matrix[i][0] > max_score:
            max_score = matrix[i][0]
            error = matrix[i][1]
            idx = i
    return idx, max_score, error

In [6]:
def get_best_layer(model_identifier, scr_perturb=None):
    
    best_layer_dict = {}
    
    for filename in os.listdir(working_dir):
        if os.path.isdir(os.path.join(working_dir,filename)):
            continue
        if 'nouns-delete50percent' in filename:
            continue
        include_condition = False
        model_name = filename.split(",")[1]
        if "model=" + model_identifier == model_name:

            if (f"-encoding-{scr_perturb}" in filename) or ("-encoding-scrambled-original" in filename):
                include_condition = True

            if include_condition == True:
                condition = filename.split(",")[0].split("Pereira2018-encoding-")[-1]

                if scr_perturb == "perturb":
                    condition = re.sub("perturb-","",condition)
                    condition = re.sub("scrambled-","",condition)

                elif scr_perturb == "scrambled":
                    if not any(x in condition for x in ["1", "3", "5", "7"]):
                        condition = re.sub("scrambled-","",condition)
                    else:
                        condition = condition

                elif scr_perturb == "control":
                    condition = re.sub("scrambled-","",condition)

                file = os.path.join(working_dir,filename)
                with open(file, 'rb') as f:
                    out = pickle.load(f)
                result = out['data'].values
                
                layers = list(out["data"].layer.values)
                idx, max_score, error = get_max_score(result)
                best_layer = layers[idx]
                
                best_layer_dict[condition] = best_layer
    return best_layer_dict

`gpt2`

In [7]:
best_layer_control = get_best_layer(model_identifier="gpt2", scr_perturb="control")
best_layer_control

{'original': 'encoder.h.10'}

In [8]:
best_layer_perturb = get_best_layer(model_identifier="gpt2", scr_perturb="perturb")
best_layer_perturb

{'contentwords': 'encoder.h.11',
 'functionwords': 'encoder.h.11',
 'nouns': 'encoder.h.11',
 'nounsverbs': 'encoder.h.11',
 'nounsverbsadj': 'encoder.h.11',
 'random-nouns': 'encoder.h.11',
 'random-nouns-controlled': 'encoder.h.1',
 'sentenceshuffle_passage': 'encoder.h.11',
 'sentenceshuffle_random': 'encoder.h.11',
 'sentenceshuffle_random-topic-criteria': 'encoder.h.11',
 'sentenceshuffle_random-topic-length-criteria': 'encoder.h.11',
 'sentenceshuffle_topic': 'encoder.h.11',
 'original': 'encoder.h.10'}

In [9]:
best_layer_scrambled = get_best_layer(model_identifier="gpt2", scr_perturb="scrambled")
best_layer_scrambled

{'backward': 'encoder.h.11',
 'lowpmi': 'encoder.h.11',
 'lowpmi-random': 'encoder.h.11',
 'original': 'encoder.h.10',
 'random-wl': 'encoder.h.11',
 'random-wl-samepos': 'encoder.h.11',
 'scrambled1': 'encoder.h.11',
 'scrambled3': 'encoder.h.11',
 'scrambled5': 'encoder.h.11',
 'scrambled7': 'encoder.h.11'}

In [10]:
# gather into table

manipulation = []
best_layer = []
category = []

for ind, dictionary in enumerate([best_layer_perturb, best_layer_scrambled, best_layer_control]):
    manipulation += list(dictionary.keys())
    best_layer += list(dictionary.values())
    if ind == 0:
        t = [["perturb"]*len(dictionary)]
    elif ind == 1:
        t = [["scrambled"]*len(dictionary)]
    else:
        t = [["control"]*len(dictionary)]
    flat_list = [item for sublist in t for item in sublist]
    category += flat_list
        
best_layer_df = pd.DataFrame({
    "manipulation_category" : category,
    "manipulation": manipulation,
    "best_layer" : best_layer
})

best_layer_df

,manipulation_category,manipulation,best_layer
0,perturb,contentwords,encoder.h.11
1,perturb,functionwords,encoder.h.11
2,perturb,nouns,encoder.h.11
3,perturb,nounsverbs,encoder.h.11
4,perturb,nounsverbsadj,encoder.h.11
5,perturb,random-nouns,encoder.h.11
6,perturb,random-nouns-controlled,encoder.h.1
7,perturb,sentenceshuffle_passage,encoder.h.11
8,perturb,sentenceshuffle_random,encoder.h.11
9,perturb,sentenceshuffle_random-topic-criteria,encoder.h.11


In [11]:
best_layer_df.to_csv("best_layer_per_condition_gpt2.csv",index=None)

`gpt2-avgtoken`

In [12]:
best_layer_control_avg = get_best_layer(model_identifier="gpt2", scr_perturb="control")
best_layer_control_avg

{'original': 'encoder.h.10'}

In [13]:
best_layer_perturb_avg = get_best_layer(model_identifier="gpt2", scr_perturb="perturb")
best_layer_perturb_avg

{'contentwords': 'encoder.h.11',
 'functionwords': 'encoder.h.11',
 'nouns': 'encoder.h.11',
 'nounsverbs': 'encoder.h.11',
 'nounsverbsadj': 'encoder.h.11',
 'random-nouns': 'encoder.h.11',
 'random-nouns-controlled': 'encoder.h.1',
 'sentenceshuffle_passage': 'encoder.h.11',
 'sentenceshuffle_random': 'encoder.h.11',
 'sentenceshuffle_random-topic-criteria': 'encoder.h.11',
 'sentenceshuffle_random-topic-length-criteria': 'encoder.h.11',
 'sentenceshuffle_topic': 'encoder.h.11',
 'original': 'encoder.h.10'}

In [14]:
best_layer_scrambled_avg = get_best_layer(model_identifier="gpt2", scr_perturb="scrambled")
best_layer_scrambled_avg

{'backward': 'encoder.h.11',
 'lowpmi': 'encoder.h.11',
 'lowpmi-random': 'encoder.h.11',
 'original': 'encoder.h.10',
 'random-wl': 'encoder.h.11',
 'random-wl-samepos': 'encoder.h.11',
 'scrambled1': 'encoder.h.11',
 'scrambled3': 'encoder.h.11',
 'scrambled5': 'encoder.h.11',
 'scrambled7': 'encoder.h.11'}